<a href="https://colab.research.google.com/github/Bossism/DailyStudy/blob/master/MessagePassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cpu.html

In [3]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree


class GCNConv(MessagePassing):

    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add', flow='source_to_target')
        # "Add" aggregation (Step 5).
        # flow='source_to_target' 表示消息从源节点传播到目标节点
        self.lin = torch.nn.Linear(in_channels, out_channels)


    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        
        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)


    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j


In [4]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='dataset', name='Cora')
data = dataset[0]
net = GCNConv(data.num_features, 64)
h_nodes = net(data.x, data.edge_index)
print(h_nodes.shape)

Processing...


torch.Size([2708, 64])


Done!


In [5]:
class My_GCN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(My_GCN, self).__init__(aggr='mean')
        self.lin = torch.nn.Linear(in_channels, out_channels)
    
    def forward(self, x, edge_index):
        # 添加自环
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # 将输入的节点特征进行线性变换
        x = self.lin(x)
        
        # 计算归一化系数norm
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        # 执行消息传递
        return self.propagate(edge_index, x=x, norm=norm, deg=deg.view(-1,1))
    
    def message(self, x_j, norm, deg_i):
        # 利用计算得到的归一化系数norm来归一化节点特征
        return norm.view(-1,1) * x_j * 2 * deg_i


In [6]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='dataset/Cora', name='Cora')
dataset

Processing...
Done!


Cora()

In [7]:
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [8]:
Net = My_GCN(dataset.num_features,dataset.num_classes)
Net

My_GCN(
  (lin): Linear(in_features=1433, out_features=7, bias=True)
)

In [9]:
out_nodes_features = Net(data.x, data.edge_index)
out_nodes_features.shape

torch.Size([2708, 7])

In [10]:
pred = out_nodes_features.argmax(dim=1)
pred

tensor([1, 5, 1,  ..., 5, 5, 0])

In [11]:
Accuracy = int((pred[data.test_mask] == data.y[data.test_mask]).sum()) / data.test_mask.sum()
Accuracy = Accuracy.item()
Accuracy

0.11900000274181366